# <b style='color:blue;'> Random Forest </b>
* Este Jupyter tem como função treinar um `Random Forest` com os melhores parametros determinados por um `RandomSearch`

-> Treinamento sem pca

---

Bibliotecas/Módulos

In [1]:
import logging
import numpy as np
import pandas as pd
import pickle as pkl
import sys

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split

Configurações dos logs

In [2]:
logger = logging.getLogger("SVM")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - [+] ------- %(message)s -------') 
handler.setFormatter(formatter)
logger.handlers = [handler]

Cores

In [3]:
RED = "\033[1;31m"
BLUE = "\033[1;34m"
GREEN = "\033[1;32m"
PINK = "\033[1;45m"
HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
MAG = "\033[1;45m"

---

## Preparando os dados

In [4]:
df = pd.read_csv("dados_FEM_MAS_10_voluntarios_s_semanas.csv")

In [5]:
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,label
11793,-1,3,-5,-7,7,4,1,6,-5,1,...,2,-6,1,1,1,3,0,8,8,1


Verificando tamanho do dataset

In [6]:
len(df)

53760

* Seleção da variável target e das features

In [7]:
y = df["label"]

In [8]:
X = df.drop(["label"], axis=1)

Separando os dados em dados de treino e teste

In [9]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=7)

---

## Treinamento

### RandomSeach

Parametros do modelo escolhidos para otimização

In [ ]:
values = {'n_estimators':[1, 100, 1000], 'min_samples_leaf': [1, 3], 'n_jobs':[1]}

In [ ]:
clf = RandomForestClassifier()

In [ ]:
def best_params(values: dict, clf: 'classificador', x_treino: np.array, x_teste: np.array, y_treino: np.array, y_teste: np.array) -> list:
    '''
        Instancia um classificador de busca e procura os melhores parâmetros para o modelo.
        
        Params
        ------
        :values: parametros a serem testados
        :clf: instancia do classificador desejado
        :x_treino: dados de treino
        :x_teste: dados de teste
        :y_treino: dados de treino -labels
        :y_teste: dados de treino - labels
        
        Return
        ------
        :best_params: lista com os melhores parametros
    '''
    
    logger.info('Instanciando RF')
    random_clf = RandomizedSearchCV(clf, param_distributions=values, n_iter=200)
    
    logger.info('Treinando RF')
    random_clf.fit(x_treino, y_treino)
    
    logger.info('Predict RF')
    y_random_clf_rl = random_clf.predict(x_teste)
    #print(y_random_clf_rl)
    
    logger.info('Parametros RF')
    print(RED, random_clf.get_params())
    
    logger.info('Best Params RF')
    best_params = random_clf.best_params_
    print(BLUE, best_params)
    
    return best_params

In [ ]:
%%time

try:
    logging.info("teste")
    best = best_params(values, clf, x_treino, x_teste, y_treino, y_teste)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Busca finalizada')

Melhores parâmetros:

{'n_jobs': 1, 'n_estimators': 1000, 'min_samples_leaf': 1}

In [ ]:
best

----

### Random Forest

In [10]:
def classificador(x_treino: np.array, y_treino: np.array, x_teste: np.array, y_teste: np.array):
    '''
        Treina um classificador com os melhores parametros possíveis.
        
        Params
        ------
        :x_treino: dados de treino
        :x_teste: dados de teste
        :y_treino: dados de treino -labels
        :y_teste: dados de treino - labels
    
        Return
        ------
        :RF: Modelo Random Forest treinado
        :resultados (matrix): métricas
    '''
    
    logger.info('Instanciando Random Forest')
    RF = RandomForestClassifier(n_jobs= 1, n_estimators= 1000, min_samples_leaf= 1)
    
    logger.info('Treinando Random Forest')
    RF.fit(x_treino, y_treino)
    
    logger.info('Predict')
    p = RF.predict_proba(x_teste)[:, 1]
    categorias = RF.predict(x_teste)
    
    print('Métricas: \n \n')
    
    logger.info('Roc')
    roc = roc_auc_score(y_teste, p)
    print(OKBLUE, "Roc \n" + str(roc) + "\n")
    
    logger.info('Acurácia')
    acc = accuracy_score(y_teste, categorias)
    print(OKCYAN, "Acc \n" + str(acc) + "\n")
    
    logger.info("Mcc")
    mcc = matthews_corrcoef(y_teste, categorias)
    print(WARNING, "Mcc \n" + str(mcc) + "\n")
    
    logger.info('Class Report')
    resultados = classification_report(y_teste, categorias)
    print(ENDC, "Class Report \n" + resultados + "\n")
    
    return RF, resultados

In [11]:
%%time

try:
    RF, resultados = classificador(
                        x_treino,
                        y_treino,
                        x_teste,
                        y_teste)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Treino finalizado')

SVM - INFO - [+] ------- Instanciando Random Forest -------
SVM - INFO - [+] ------- Treinando Random Forest -------
SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Ok -------
SVM - INFO - [+] ------- Treino finalizado -------


Métricas: 
 

 Roc 
0.9146942047017856

 Acc 
0.7965029761904762

 Mcc 
0.5932775403371121

 Class Report 
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      5372
           1       0.81      0.78      0.79      5380

    accuracy                           0.80     10752
   macro avg       0.80      0.80      0.80     10752
weighted avg       0.80      0.80      0.80     10752


CPU times: user 30min 51s, sys: 0 ns, total: 30min 51s
Wall time: 30min 51s


----

#### Exportando modelo

In [12]:
pkl.dump(RF, open('modelos/rf_s_pca_dados_FEM_MAS_10_voluntarios_s_semanas_15_sinais_noout.pkl', 'wb'))